"Feature-based transfer learning” in NLP.

* Pre-trained DL model = feature extractor.

* ML algorithm = downstream task solver.

HYBRID: It’s not “pure deep learning” (since you’re not training/fine-tuning the whole transformer end-to-end).

It’s also not “pure machine learning” (since your features are learned by a deep neural network, not hand-crafted).

### SBERT + Logistic Regression = Deep learning for embeddings + Machine learning for classification → a hybrid approach that’s especially effective for small/medium labeled datasets. 

In [1]:
import pandas as pd

# Loading the dataset
path = "/Users/gozde/code/g0zzy/stress_sense/raw_data/Data.csv"
data = pd.read_csv(path)

data.drop(columns=["Unnamed: 0"], inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

In [2]:
import re

def strip_urls(text: str) -> str:
    """
    Remove URLs (http, https, www, youtu links) from a string.
    """
    # remove http/https URLs
    text = re.sub(r"http\S+", "", text)
    # remove www.* URLs
    text = re.sub(r"www\.\S+", "", text)
    # remove youtube short links
    text = re.sub(r"youtu\.be\S+", "", text)
    return text.strip()

## Cleaned data

In [3]:
data.statement = data.statement.apply(strip_urls)

## Only classify stress, anxiety and normal

In [ ]:
from collections import Counter
from sklearn.model_selection import train_test_split


TARGET = {"Stress", "Anxiety", "Normal"}

# 1) Filter to the 3 classes
df = data.dropna(subset=["statement","status"])
df = df[df["status"].isin(TARGET)].reset_index(drop=True)

texts  = df["statement"].tolist()
labels = df["status"].tolist()
print(Counter(labels))

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)


Counter({'Normal': 16040, 'Anxiety': 3623, 'Stress': 2296})


In [12]:
from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Encode (normalize for cosine/logreg stability)
E_train = sbert.encode(X_train, normalize_embeddings=True, show_progress_bar=True)
E_test  = sbert.encode(X_test,  normalize_embeddings=True, show_progress_bar=True)

Batches: 100%|██████████| 138/138 [00:04<00:00, 27.89it/s]


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc  = le.transform(y_test)

clf = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",      # helpful when classes are imbalanced
    multi_class="auto",
    n_jobs=-1
)
clf.fit(E_train, y_train_enc)

pred = clf.predict(E_test)
print("Accuracy:", accuracy_score(y_test_enc, pred))
print(classification_report(y_test_enc, pred, target_names=le.classes_))

/Users/gozde/.pyenv/versions/3.10.6/envs/stress_sense/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.9043715846994536
              precision    recall  f1-score   support

     Anxiety       0.83      0.87      0.85       725
      Normal       0.99      0.93      0.96      3208
      Stress       0.60      0.81      0.69       459

    accuracy                           0.90      4392
   macro avg       0.80      0.87      0.83      4392
weighted avg       0.92      0.90      0.91      4392



### 👆 Probably also good enough for our use case. 